### portmy database: stocks table
### stock database: buy, price tables

In [1]:
import pandas as pd
import numpy as np
from datetime import date, timedelta
from sqlalchemy import create_engine

engine = create_engine("sqlite:///c:\\ruby\\portmy\\db\\development.sqlite3")
conmy = engine.connect()
conmy = engine.connect()
engine = create_engine("mysql+pymysql://root:@localhost:3306/stock")
const = engine.connect()

pd.set_option('display.float_format','{:,.2f}'.format)

data_path = "../data/"
csv_path = "\\Users\\User\\iCloudDrive\\"
box_path = "\\Users\\User\\Dropbox\\"

today = date.today()
week_str = today.strftime('%Y-%U')
today, week_str

(datetime.date(2022, 5, 21), '2022-20')

### Process after last business day, today must be last business day

In [2]:
today = today - timedelta(days=1)
wkago = today - timedelta(days=7)
today, wkago

(datetime.date(2022, 5, 20), datetime.date(2022, 5, 13))

### Restart & Run All Cells

### This process affects only already in port stocks. To highlight price changes in percent.

In [3]:
cols = 'name price_d price_w percent perform mrkt '.split()

In [4]:
format_dict = {
    'qty':'{:,}',    
    'price':'{:.2f}','price_d':'{:.2f}','price_w':'{:.2f}',
    'max_price':'{:.2f}','min_price':'{:.2f}',
    'maxp':'{:.2f}','minp':'{:.2f}','opnp':'{:.2f}',    
    'pct':'{:,.2f}%','percent':'{:,.2f}%',
    
    'pe':'{:.2f}','pbv':'{:.2f}',
    'paid_up':'{:,.2f}','market_cap':'{:,.2f}',
    'daily_volume':'{:,.2f}','beta':'{:,.2f}', 
    'dly_vol':'{:,.2f}', 
}

In [5]:
sql = """
SELECT name, date, price 
FROM buy 
ORDER BY date DESC 
LIMIT 1
"""
df_buy = pd.read_sql(sql, const)
df_buy.style.format(format_dict)

,name,date,price
0,DCC,2022-05-05,2.96


In [6]:
names = df_buy['name']
name = names.to_string(index=False)

sql = '''
SELECT * FROM stocks WHERE name = '%s'
'''
sql = sql % name
print(sql)

df_stocks = pd.read_sql(sql, conmy)
df_stocks.style.format(format_dict) 


SELECT * FROM stocks WHERE name = 'DCC'



,id,name,market,price,max_price,min_price,pe,pbv,paid_up,market_cap,daily_volume,beta,ticker_id,created_at,updated_at
0,135,DCC,SET,2.80,3.40,2.70,14.67,4.32,999.99,"25,551.71","47,107.94",0.06,135,2017-07-23 07:24:36.765074,2022-05-21 08:59:30.634760


In [7]:
sql = '''
SELECT * FROM price WHERE name = "%s" ORDER BY date DESC LIMIT 5
'''
sql = sql % name
print(sql)

df_price = pd.read_sql(sql, const)
df_price.style.format(format_dict)


SELECT * FROM price WHERE name = "DCC" ORDER BY date DESC LIMIT 5



,name,date,price,maxp,minp,qty,opnp
0,DCC,2022-05-20,2.80,2.82,2.76,"16,903,539",2.76
1,DCC,2022-05-19,2.74,2.78,2.74,"46,973,487",2.76
2,DCC,2022-05-18,2.80,2.84,2.78,"11,276,667",2.84
3,DCC,2022-05-17,2.80,2.80,2.72,"21,061,451",2.72
4,DCC,2022-05-13,2.74,2.80,2.70,"42,871,901",2.80


In [8]:
sql = '''
SELECT name
FROM buy 
WHERE active = 1
ORDER BY name'''
df_price = pd.read_sql(sql, const)

names = df_price.name.tolist()
in_p = ", ".join(map(lambda name: "'%s'" % name, names))

sql = """
SELECT name, price AS price_d 
FROM price 
WHERE date = '%s' AND name IN (%s)
ORDER BY name, date"""
sql = sql % (today, in_p)
print(sql)

df_today = pd.read_sql(sql, const)
df_today.shape[0]


SELECT name, price AS price_d 
FROM price 
WHERE date = '2022-05-20' AND name IN ('ASP', 'BCH', 'DCC', 'DIF', 'DOHOME', 'HREIT', 'IP', 'JASIF', 'KCE', 'MAKRO', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'RJH', 'SCC', 'SCCC', 'SENA', 'STA', 'SYNEX', 'TISCO', 'TMT', 'TPIPL', 'WHART')
ORDER BY name, date


25

In [9]:
sql = """
SELECT name, price AS price_w
FROM price 
WHERE date = '%s' AND name IN (%s) 
ORDER BY name"""
sql = sql % (wkago, in_p)
print(sql)

df_wkago = pd.read_sql(sql, const)
df_wkago.shape[0]


SELECT name, price AS price_w
FROM price 
WHERE date = '2022-05-13' AND name IN ('ASP', 'BCH', 'DCC', 'DIF', 'DOHOME', 'HREIT', 'IP', 'JASIF', 'KCE', 'MAKRO', 'MCS', 'NER', 'ORI', 'PTTGC', 'RCL', 'RJH', 'SCC', 'SCCC', 'SENA', 'STA', 'SYNEX', 'TISCO', 'TMT', 'TPIPL', 'WHART') 
ORDER BY name


25

In [10]:
trend = pd.merge(df_today, df_wkago, on='name',how='inner')
trend.shape

(25, 3)

In [11]:
def performance(vals):
    price_d, price_w = vals
    if (price_d > price_w):
        return 'Better'
    elif (price_d < price_w):
        return 'Worse'
    else:
        return 'No Change'

In [12]:
trend['percent'] = (trend.price_d-trend.price_w)/trend.price_w * 100

In [13]:
trend

,name,price_d,price_w,percent
0,ASP,3.16,3.08,2.60
1,BCH,18.80,20.00,-6.00
2,DCC,2.80,2.74,2.19
3,DIF,14.00,14.30,-2.10
4,DOHOME,19.10,18.80,1.60
5,HREIT,8.45,8.35,1.20
6,IP,20.50,18.30,12.02
7,JASIF,10.90,11.00,-0.91
8,KCE,63.25,61.75,2.43
9,MAKRO,36.50,34.00,7.35


In [14]:
trend["perform"] = trend[["price_d","price_w"]].apply(performance, axis=1)

In [15]:
trend.sort_values(by=['percent'],ascending=[True]).style.format(format_dict)

,name,price_d,price_w,percent,perform
1,BCH,18.80,20.00,-6.00%,Worse
3,DIF,14.00,14.30,-2.10%,Worse
7,JASIF,10.90,11.00,-0.91%,Worse
24,WHART,10.60,10.60,0.00%,No Change
21,TISCO,89.50,89.25,0.28%,Better
18,SENA,3.94,3.92,0.51%,Better
22,TMT,9.20,9.15,0.55%,Better
15,RJH,33.25,33.00,0.76%,Better
16,SCC,361.00,358.00,0.84%,Better
5,HREIT,8.45,8.35,1.20%,Better


In [16]:
trend.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,84.00%
Worse,12.00%
No Change,4.00%


In [17]:
sql = '''
SELECT name, max_price AS max, min_price AS min, pe, pbv, daily_volume AS volume, beta, market
FROM stocks
'''
my_stocks = pd.read_sql(sql, conmy)
my_stocks.shape

(252, 8)

In [18]:
filters = [
   (my_stocks.market.str.contains('SET50')),
   (my_stocks.market.str.contains('SET100')),
   (my_stocks.market.str.contains('mai'))]
values = ['SET50','SET100','mai']
my_stocks["mrkt"] = np.select(filters, values, default='SET999')

In [19]:
trend2 = pd.merge(trend, my_stocks, on='name', how='inner')
trend2.sort_values(['percent'],ascending=[True]).shape

(25, 13)

In [20]:
set50 = trend2.mrkt.str.contains('SET50') 
flt_set50 = trend2[set50]
flt_set50[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
21,TISCO,89.50,89.25,0.28%,Better,SET50
16,SCC,361.00,358.00,0.84%,Better,SET50
8,KCE,63.25,61.75,2.43%,Better,SET50
13,PTTGC,46.50,44.25,5.08%,Better,SET50


In [21]:
flt_set50.perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,100.00%


In [22]:
set100 = trend2.mrkt.str.contains('SET100')
flt_set100 = trend2[set100]
flt_set100[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
1,BCH,18.80,20.00,-6.00%,Worse,SET100
4,DOHOME,19.10,18.80,1.60%,Better,SET100
14,RCL,46.75,45.50,2.75%,Better,SET100
19,STA,23.80,22.80,4.39%,Better,SET100
20,SYNEX,21.70,20.60,5.34%,Better,SET100
12,ORI,9.80,9.30,5.38%,Better,SET100


In [23]:
flt_set100[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,83.33%
Worse,16.67%


In [24]:
set999 = trend2.mrkt.str.contains('SET999')
flt_set999 = trend2[set999]
flt_set999[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
3,DIF,14.00,14.30,-2.10%,Worse,SET999
7,JASIF,10.90,11.00,-0.91%,Worse,SET999
24,WHART,10.60,10.60,0.00%,No Change,SET999
18,SENA,3.94,3.92,0.51%,Better,SET999
22,TMT,9.20,9.15,0.55%,Better,SET999
15,RJH,33.25,33.00,0.76%,Better,SET999
5,HREIT,8.45,8.35,1.20%,Better,SET999
17,SCCC,149.50,147.50,1.36%,Better,SET999
11,NER,6.60,6.50,1.54%,Better,SET999
2,DCC,2.80,2.74,2.19%,Better,SET999


In [25]:
flt_set999[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,78.57%
Worse,14.29%
No Change,7.14%


In [26]:
setmai = trend2.mrkt.str.contains('mai')
flt_setmai = trend2[setmai]
flt_setmai[cols].sort_values(by=['percent','name'],ascending=[True,True]).style.format(format_dict)

,name,price_d,price_w,percent,perform,mrkt
6,IP,20.50,18.30,12.02%,Better,mai


In [27]:
flt_setmai[cols].perform.value_counts(normalize=True).to_frame().style.format('{:.2%}')

,perform
Better,100.00%
